# CD7 gating cells of interest

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from scip_workflows.common import *

In [ ]:
import flowutils
from matplotlib.patches import Polygon
from matplotlib.path import Path as mPath
from sklearn.preprocessing import scale

from scip_workflows.core import plot_gate_czi
from scip_workflows.masking import plot_scip_segmentation


In [ ]:
data_dir = Path(
    "/data/gent/vo/000/gvo00070/vsc42015/datasets/cd7/800/results/scip/202203221745/"
)

df = pq.read_table(data_dir / "features.parquet").to_pandas()

df = df.set_index(["meta_panel", "meta_replicate", "meta_P", "meta_id"])
df = df.loc["D"]
df = df.loc[numpy.load(data_dir / "index.npy", allow_pickle=True)]
df = df.sort_index()


In [ ]:
df.shape


# Gating

In [ ]:
def asinh_scale(x, t):
    return scale(
        flowutils.transforms.asinh(x, channel_indices=None, t=t, m=4.5, a=1),
        with_std=False,
    )


In [ ]:
fluor_df = df.groupby(["meta_replicate", "meta_P"])[
    ["feat_sum_DAPI", "feat_sum_RPe", "feat_sum_EGFP", "feat_sum_APC"]
].transform(lambda x: asinh_scale(x, df["feat_sum_" + x.name.split("_")[-1]].max()))


In [ ]:
melted_df = pandas.melt(
    fluor_df.reset_index(),
    id_vars=["meta_P", "meta_replicate"],
    value_vars=fluor_df.columns,
)
grid = seaborn.FacetGrid(
    data=melted_df, col="meta_replicate", row="variable", sharey=True, aspect=1.5
)
grid.map_dataframe(seaborn.stripplot, x="meta_P", y="value", size=1, alpha=0.5)
grid.add_legend()


## DAPI

In [ ]:
seaborn.histplot(data=fluor_df.reset_index(), x="feat_sum_DAPI")


In [ ]:
sel1 = fluor_df["feat_sum_DAPI"] < -0.1
plot_gate_czi(sel1, df, maxn=15, channel=0)


In [ ]:
sel1 = fluor_df["feat_sum_DAPI"] < 0.1
sel2 = fluor_df["feat_sum_DAPI"] > -0.1
plot_gate_czi(sel1 & sel2, df, maxn=15, channel=0)


## EGFP (CD45)

In [ ]:
seaborn.histplot(data=fluor_df.reset_index(), x="feat_sum_EGFP")


### APC (CD15)

In [ ]:
seaborn.displot(data=fluor_df.reset_index(), x="feat_sum_APC")


In [ ]:
vertices = numpy.array(
    [
        [-0.03, 0.02],
        [-0.01, 0.07],
        [0.025, 0.08],
        [0.1, 0.055],
        [0.1, 0.0],
        [0.01, 0.0],
        [-0.025, -0.015],
    ]
)
p = Polygon(vertices, facecolor="none", edgecolor="red")

fig, ax = plt.subplots(dpi=150)
seaborn.scatterplot(
    data=fluor_df,
    x="feat_sum_EGFP",
    y="feat_sum_APC",
    hue="feat_sum_DAPI",
    palette="viridis",
    norm=tuple(df["feat_sum_DAPI"].quantile((0.05, 0.95))),
    s=0.5,
    alpha=0.5,
    edgecolor="none",
    ax=ax,
)
ax.add_patch(p)
seaborn.despine(fig)


In [ ]:
path = mPath(vertices)
idx = path.contains_points(fluor_df[["feat_sum_EGFP", "feat_sum_APC"]])


In [ ]:
sum(idx) / len(idx)


In [ ]:
vertices = numpy.array([[0.0, -0.01], [0.07, 0.06], [0.07, -0.01]])
p = Polygon(vertices, facecolor="none", edgecolor="red")

fig, ax = plt.subplots(dpi=150)
seaborn.scatterplot(
    data=fluor_df[idx],
    x="feat_sum_RPe",
    y="feat_sum_APC",
    s=0.5,
    alpha=0.5,
    edgecolor="none",
    ax=ax,
)
ax.add_patch(p)
seaborn.despine(fig)


In [ ]:
path = mPath(vertices)
idx2 = ~path.contains_points(fluor_df[idx][["feat_sum_APC", "feat_sum_RPe"]])


In [ ]:
# fraction neutrophils
len(fluor_df[idx][idx2]) / len(fluor_df)


In [ ]:
# fraction eosinophils
len(fluor_df[idx][~idx2]) / len(fluor_df)


In [ ]:
len(fluor_df[idx][idx2])


In [ ]:
len(fluor_df[idx][~idx2])


In [ ]:
numpy.save(data_dir / "neutrophils.npy", fluor_df[idx][idx2].index)


In [ ]:
vertices = numpy.array(
    [[0.05, -0.01], [-0.05, -0.07], [-0.05, -0.05], [0, -0.005], [0.03, 0]]
)
p = Polygon(vertices, facecolor="none", edgecolor="red")

fig, ax = plt.subplots(dpi=150)
seaborn.scatterplot(
    data=fluor_df,
    x="feat_sum_EGFP",
    y="feat_sum_APC",
    hue="feat_sum_DAPI",
    palette="viridis",
    norm=tuple(df["feat_sum_DAPI"].quantile((0.05, 0.95))),
    s=0.5,
    alpha=0.5,
    edgecolor="none",
    ax=ax,
)
ax.add_patch(p)
seaborn.despine(fig)


In [ ]:
path = mPath(vertices)
idx3 = path.contains_points(fluor_df[["feat_sum_EGFP", "feat_sum_APC"]])


In [ ]:
sum(idx3) / len(fluor_df)


In [ ]:
vertices = numpy.array(
    [
        [0.05, -0.07],
        [-0.05, -0.12],
        [-0.05, -0.08],
        [0, -0.04],
        [0.06, -0.01],
    ]
)
p = Polygon(vertices, facecolor="none", edgecolor="red")

fig, ax = plt.subplots(dpi=150)
seaborn.scatterplot(
    data=fluor_df,
    x="feat_sum_EGFP",
    y="feat_sum_APC",
    hue="feat_sum_DAPI",
    palette="viridis",
    norm=tuple(df["feat_sum_DAPI"].quantile((0.05, 0.95))),
    s=0.5,
    alpha=0.5,
    edgecolor="none",
    ax=ax,
)
ax.add_patch(p)
seaborn.despine(fig)


In [ ]:
path = mPath(vertices)
idx4 = path.contains_points(fluor_df[["feat_sum_EGFP", "feat_sum_APC"]])


In [ ]:
sum(idx4) / len(fluor_df)
